In [26]:
import pandas as pd
import sqlalchemy as sa
import yfinance as yf
import sys
from datetime import datetime

connection_url = sa.engine.URL.create(
    "mssql",
    host="DATOR",
    database="Finance",
    query={
        "driver": "SQL Server",
        "TrustServerCertificate": "yes",
        #"authentication": "ActiveDirectoryIntegrated",
    },
)

engine = sa.create_engine(connection_url)


In [30]:
def download_history(ticker):
    t = yf.Ticker(ticker)
    hist = t.history(period='max', interval='1d')
    hist['Date'] = hist.index.strftime('%Y-%m-%d')
    hist['Ticker'] = ticker
    hist['Timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    hist = hist.reset_index(drop=True)
    hist.to_sql('History', if_exists='append', index=False, con=engine, chunksize=1000)
    


In [31]:
tickers = ('MSFT', 'DE', 'AAPL', 'ORCL', 'CAT', 'VOLV-B.ST')

for t in tickers:
    download_history(t)
    